In [6]:
!pip install python-dotenv

In [24]:

from openai import OpenAI
client = OpenAI()
def ask_llm(prompt, model = 'gpt-5-nano', temp = 1):        # 'gpt-5-nano 은 temp를 조절할 수 없음
    response = client.chat.completions.create(
        model = model,
        temperature = temp,     # temperature은 창의성의 정도를 설정하는것. 낮을수록 자유도가 낮음
        messages = [{
            'role' : 'user',
            'content' : prompt}])
    return response.choices[0].message.content

---

<span style="color: Gold"> 1. Zero-shot prompt

In [27]:
# 1. zero-shot prompt
# 예시없이 지시사항만 던지는 것 - LLM 기본기능
prompt = "이 문장의 감정을 분류해: '오늘 점심 메뉴가 품절이라 너무 슬퍼.'"
print(ask_llm(prompt))

감정 분류: 부정적 감정 → 슬픔 (강도 높음)


<span style="color: Gold"> 2. few-shot prompting

In [28]:
#2. few-shot prompting
# 이렇게 하는거야 하고 예시(shot)를 몇개 보여줘서 성능을 높이는 기술
prompt = """
단어를 이모지로 바꿔줘
사과 -> 🍎
자동차 -> 🚗
고양이 -> 🐱
비행기 ->
"""
print(ask_llm(prompt, temp = 1))

완료된 매핑:
- 사과 -> 🍎
- 자동차 -> 🚗
- 고양이 -> 🐱
- 비행기 -> ✈️

다른 단어도 필요하면 말해줘!


In [ ]:
# 3. chain-of-thought-prompting cot(생각의 사슬)
prompt = """
질문 : 5개의 사과 중 2개를 먹고 3개를 더 샀어. 몇개 남았지
사과의 단가는 100원: 1000원
총 지출금액은 얼마지
답변하기 전에 단계별로 생각해서 과정을 적어줘
"""
print(ask_llm(prompt, temp = 1))

# 총 남은 사과의 개수를 세고 사과를 구입할때 드는 비용을
# 계산해서 거스름돈을 계산해줘
# 계산은 먹은 사과와 남은 사과를 모두 포함한 금액
# 머자막 출력은 전체 로직을 점검해서 오류가 있는지 확인하고 결과를 알려줘


요청하신 “단계별 사고 과정”은 내부 생각까지 자세히 보여주지는 못하지만, 필요한 계산은 간략하게 정리해 드리겠습니다.

계산 요약
- 남은 사과 수
  - 처음 5개에서 2개 먹음 → 3개 남음
  - 그다음 3개를 더 샀음 → 총 6개
- 사과 1개당 가격: 100원
- 총 비용(6개): 6 × 100 = 600원
- 거스름돈
  - 보통 1000원을 지불했다고 가정하면 거스름돈 = 1000 - 600 = 400원

참고
- 문제에 “총 지출금액: 1000원”이 제시되어 있는데, 이 값이 실제로 사과에 쓴 돈이 맞는지(600원과 다른지) 애매합니다. 만약 1000원이 실제 지출액이라면 거스름돈은 다르게 판단될 수 있습니다. 정확한 상황에 따라 거스름돈이 달라질 수 있으니, 지출액의 의도를 확인해 주세요.

정리하면:
- 남은 사과: 6개
- 총 비용: 600원
- 거스름돈(1000원으로 결제했다고 가정): 400원


In [3]:
# 4. self-consistency(자기 일관성)
# 한번만 묻지 않고 여러번(예 : 3번) 물어본 뒤 가장 많이 나온 답을 채택함
question = '철수는 학교까지 10분 걸려, 왕복은 몇 분 걸릴까?'
answer = []
for _ in range(3):
    answer.append(ask_llm(question))
print(f'수집된 답변들: ', answer)
from collections import Counter
counter = Counter(answer)
counter

수집된 답변들:  ['20분. 한 방향이 10분이니까 왕복은 10+10=20분으로 가정(같은 속도로 가고 대기 없이 왕복하는 경우).', '20분 걸립니다. 편도 10분이므로 왕복은 10분 + 10분 = 20분입니다.', '20분입니다.  \n편도 이동에 10분이 걸리므로 왕복은 10분 + 10분 = 20분이 됩니다.']


Counter({'20분. 한 방향이 10분이니까 왕복은 10+10=20분으로 가정(같은 속도로 가고 대기 없이 왕복하는 경우).': 1,
         '20분 걸립니다. 편도 10분이므로 왕복은 10분 + 10분 = 20분입니다.': 1,
         '20분입니다.  \n편도 이동에 10분이 걸리므로 왕복은 10분 + 10분 = 20분이 됩니다.': 1})

In [9]:
# 5. Generate Knowlege Prompting(지식생성)
# 바로 답하지 말고 관려된 지식을 먼저 생성한 뒤 그 지식을 바탕으로 답하게 됨
# 1. 지식을 생성
knowledge = ask_llm('골프라는 스포츠에 대해 사실적인 지식 3가지 나열해줘')
print(f'[지식] {knowledge} ')
    # 2 단계 : 지식을 활용해 답변
prompt = f""" 다음 지식을 참고해서 '골프에서 홀인원이 왜 어려운지' 설명해줘."
모든 답변은 한글로 작성
[지식] : {knowledge}
"""
print(ask_llm(prompt))


[지식] 다음은 골프에 대한 사실 3가지입니다.

- 18홀 코스에서의 목표: 골프는 일반적으로 18홀로 구성된 코스에서 가능한 적은 타수로 모든 홀을 끝내는 것이 목표입니다. 한 라운드의 길이는 코스와 속도에 따라 다르지만 보통 수시간이 걸립니다.

- 한 번에 들 수 있는 클럽 수의 한도: 선수는 한 번에 들고 다닐 수 있는 클럽의 수가 14개로 정해져 있습니다. 이 규정은 플레이의 공정성을 위해 존재합니다.

- 규칙의 관리 주체와 주요 대회: 골프 규칙은 주로 미국골프협회(USGA)와 영국의 R&A가 공동으로 관리합니다. 세계적으로 유명한 메이저 대회로는 마스터스, US 오픈, 더 오픈(Open Championship, 브리티시 오픈), PGA 챔피언십이 있습니다. 
다음 지식을 바탕으로 골프에서 홀인원이 왜 어려운지 설명드릴게요.

- 핵심 Idea
  - 홀인원은 한 번의 샷으로 공이 바로 홀컵에 들어가야 하는 매우 작은 표적에 정확히 맞춰야 하는 일이라서 어렵습니다. 거리가 아무리 짧은 홀이라도, 컵은 지름이 약 4.25인치 정도로 작고 표적은 아주 미세한 오차도 허용하지 않기 때문입니다.

- 왜 어려운가의 구체적 이유
  - 거리와 정확도의 결합
    - 골프는 18홀로 구성된 코스를 돌며 각 홀의 거리와 목표선이 다릅니다. 한 번의 샷으로 정확한 거리와 직선을 맞추는 일이 쉽지 않습니다.
  - 환경의 변수들
    - 바람 방향과 세기, 땅의 경사, 그린의 속도와 경사, 잔디의 상태, 공의 스핀 등 여러 환경 요인이 함께 작용합니다. 이 모든 요소가 샷의 궤적과 최종 위치에 영향을 미쳐 홀에 직접 들어가게 하는 확률을 떨어뜨립니다.
  - 그린의 복합성
    - 그린은 빠르게 굴러가고 경사는 방향에 따라 공의 움직임이 크게 달라집니다. 설사 그린에 잘 도달했다고 해도, 공의 속도와 방향이 약간만 어긋나도 홀인원으로 이어지지 않을 수 있습니다.
  - 샷 성공률의 낮은 확률
    - 일반인(대중적 수준의 골퍼)에게 홀인원의 확률은 대략 

In [10]:

prompt = '골프에서 홀인원이 왜 어려운지 설명해줘'
print(ask_llm(prompt))


다 이유가 있지만, 핵심은 한 번의 샷으로 모든 조건이 완벽하게 맞아떨어져야 한다는 점이에요.

주요 이유

- 확률이 매우 낮다
  - 일반적인 아마추어의 평생 홀인원 확률은 대략 1만 2천 ~ 1만 5천 분의 1 정도로 추정된다.
  - 프로 선수의 par-3 홀에서의 확률은 대략 2천50~3천 분의 1 정도로 생각되곤 한다.
  - 어떤 상황에서라도 이 수치는 상황에 따라 크게 달라지지만, 전반적으로 아주 드문 일이다.

- 거리, 방향, 비행 경로를 한 번에 맞춰야 한다
  - 티샷으로 홀에 직접 들어가려면 원하는 거리로 정확하게 보내고, 정확한 방향으로 비행해야 한다.
  - 공의 비행 각도와 회전(스핀)이 미세하게만 어긋나도 홀로 들어가기가 어렵다.

- 바람, 습도, 기온, 습윤도 같은 외부 변수
  - 바람의 방향과 크기, 온도와 습도에 따른 공의 비거리 변화, 공기 저항 변화 등은 샷의 결과에 큰 영향을 준다.
  - 그린의 속도와 경사(언듈레이션) 역시 착지 후의 굴림 방향과 속도에 결정적이다.

- 홀의 크기와 구멍의 형태
  - 구멍의 직경은 약 4.25인치(약 10.8 cm). 공의 직경은 약 1.68인치(약 4.3 cm)로, 들어가려면 상당히 정밀한 경로를 따라야 한다.
  - 그린에 올라간 이후에도 직접 구멍으로 “쳐넣어” 들어가야 하는데, 이 과정에서의 미세한 차이도 결과를 바꾼다.

- 경사와 바닥 조건의 영향
  - 그린의 표면 상태, 잔디의 상태, 미세한 경사진 면들로 인해 공이 굴러 가는 방향이 예측과 다를 수 있다.
  - 만약 착지 순간의 속도가 지나치게 강하거나 약하면 즉시 구멍으로 들어가지 않거나 밖으로 튕겨나갈 수 있다.

요약하면, 홀인원은 "한 번의 샷에서 거리/방향/스핀/높이 등의 물리적 변수와 날씨·그린 상태 같은 외부 변수까지 모두 완벽히 맞아떨어지는" 드문 조합이 필요하기 때문입니다. 그래서 아주 드물고도 매력적인 샷으로 여겨집니다.

원하신다면 더 자세한 수치 예시나, 연습에서 고려할 수 있는 일반적인 

In [11]:
# 6. prompt chaining (프롬프트 체이닝)
# 복잡한 일을 한번에 시키지 않고 a작업의 결과를 b작업의 입력으로 넘겨주는 파이프라인
# step 1 : 주제 추출
text = '이메일: 안녕하세요, 이번주 금요일 회의는 2시로 변경되었습니다.'
topic = ask_llm(f'다음 텍스트에서 핵심 주제만 단어로 뽑아줘 출력은 한글로: {text}')
# step 2 : 답장 작성
reply = ask_llm(f"'{topic}에 대해 '알겠습니다'라는 정중한 답장 메일을 써줘")
print(reply)

제목: 금요일 14시 회의 변경 확인드립니다

안녕하세요, [성함/팀명]님.

금요일 14시로 회의 시간이 변경되었다는 점 잘 확인했습니다. 해당 시간에 맞춰 참석하겠습니다. 필요하신 자료나 준비물이 있다면 미리 공유해 주시면 감사하겠습니다.

알겠습니다.

감사합니다.
[이름] [직책] / [부서], [회사]
[연락처]


In [12]:
# 7. Retrival Augmented Generation (RAG 검색 증강 생성)
# 이론 : LLM이 모르는 외부 데이터(회사문서 등)을 찾아서 (retrieval)프롬프트에 넣어주고 답하게 함
# 가상의 검색된 문서
retrieved_doc = '문서내용: 우리 회사의 재택근무는 매주 수요일만 가능하다'

prompt = f'''
아래 [참조문서]를 기반으로 답변해, 문서에 없으면 모른다고 해.
[참조문서] : {retrieved_doc}
칠문 : 재택근무는 언제 할 수 있어?
'''
print(ask_llm(prompt))


재택근무는 매주 수요일에만 가능합니다. 즉, 주 1회 수요일에 할 수 있습니다.


<span style="color: Gold"> 8. Automatic Reasionig and Tool-use(ART) 자동 추론 및 도구 사용
- 모델이 스스로 어떤 도구(tool)를 쓸지 판단하고, 필요하면 연속적으로 추론(reasoning)하여 문제를 해결하는 방식


In [13]:
# 8. Automatic Reasionig and Tool-use 자동 추론 및 도구 사용
# LLM이 스스로 계산기나 검색엔진 같은 도구가 필요한지 판단하고 호출형식을 뱉어나는 것을 말함
prompt = '''
계산이 필요하면 [CALC :수식]이라고 출력해
질문 : 33452* 192 는 뭐야?
'''
response =ask_llm(prompt)
print(response)


[CALC :33452*192]
6,422,784


In [37]:
prompt = """
너는 자동 도구 선택 시스템이야.
다음과 같은 도구를 사용할 수 있어:

1.계산기 -> [CALC : 수식]
2.날씨 조회 ->[WEATHER : 도시명]
3.일반질문 - >직접입력

규칙:
- 계산이 필요하면 CALC: ... 출력
- 날씨 정보가 필요하면 WEATHER: 도시명 출력 도시명은 영문, json구조를 파싱해서 날씨정보를 읽어서
사용자에게 친화적인 답변으로 변경
- 그 외 는 일반적인 답변

질문: 3458 * 256의 결과와 
서울의 내일 날씨는 어때?
"""

def find_weather(CITY= 'Seoul'):    
    API_KEY = os.environ['OPEN_WEATHER_KEY']
    lat = 37.25
    lon = 126.45
    print(API_KEY)
    CITY = 'Seoul'
    url = f'https://api.openweathermap.org/data/2.5/weather?q={CITY}&appid={API_KEY}'
    import requests
    response = requests.get(url)
    return response.text

import re
def process_response(text):
    # 계산기
    calc = re.findall(r'CALC:\s*(.*)', text)
    if calc:
        expr = calc[0]  
        print(f'계산기 expr = {expr}')      
        return eval(expr)
    # 날씨
    weather = re.findall(r'WEATHER:\s*(.*)', text)
    if weather:
        city = weather[0]
        return find_weather(city)

response = ask_llm(prompt)
print(f'llm 출력 결과 : {response}')

# for res in response.split('\n'):
#     process_response(res)

llm 출력 결과 : CALC: 3458 * 256
WEATHER: Seoul


In [ ]:
%pip install requests

In [36]:
import os 
from dotenv import load_dotenv
load_dotenv()
API_KEY = 'fdf46d9b90a6dce13b4e54db8621e743'
lat = 37.25
lon = 126.45
print(API_KEY)
CITY = 'Seoul'
url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_KEY}'
import requests
response = requests.get(url)
response.text

fdf46d9b90a6dce13b4e54db8621e743


'{"coord":{"lon":126.45,"lat":37.25},"weather":[{"id":800,"main":"Clear","description":"clear sky","icon":"01d"}],"base":"stations","main":{"temp":287.63,"feels_like":286.99,"temp_min":287.63,"temp_max":287.63,"pressure":1016,"humidity":71,"sea_level":1016,"grnd_level":1016},"visibility":10000,"wind":{"speed":7.45,"deg":237,"gust":8.35},"clouds":{"all":0},"dt":1763955521,"sys":{"type":1,"id":8093,"country":"KR","sunrise":1763936548,"sunset":1763972371},"timezone":32400,"id":1844106,"name":"Daebudo","cod":200}'

In [23]:
# 9. automatic prompt enginer(APE)
# 사람이 프롬프트를 짜는게 아니라 LLM에게 좋은 프롬프트를 짜달라고 시키는 것
task = '고객의 리뷰에서 감정을 분석하는 작업'
prompt = f'''
나는'{task}' 을 하려고 해.
이 작업을 수행하기에 가장 완벽한 프롬프트 지시문을 바로 사용할 수 있도록 간결하게 작성해줘
'''
best_prompt = ask_llm(prompt)
print(best_prompt)

권장 프롬프트(바로 사용 가능, 간결 버전)

다음 고객 리뷰를 읽고 감정을 3단계로 분류하시오: 긍정, 부정, 중립. 가능하면 감정의 강도(score: 0.0~1.0)와 한 문장의 근거를 함께 JSON 형식으로 반환하시오. 출력 예시: { "sentiment": "긍정", "score": 0.92, "rationale": "배송이 빠르고 품질에 만족했다." }. 입력: [리뷰 텍스트]


<span style="color: Gold"> 10. Active Prompt
- LLM이 스스로 “좋은 예시(few-shot examples)”를 골라서 성능을 자동으로 높이게 하는 기술

In [34]:
# 10. active prompt
# llm이 답변하기 애매하거나 불확실한 문제를 찾아내서 사람에게 이것 좀 가르쳐 주세요(예시추가)라고 요청하는 방식
# llm에게 문제를 풀게하고 '확신도(confidence score)'를 묻는다 낮으면 그 문제를 few-shot의 예제로 추가

# 프롬프트 생성
import json
def build_prompt(task:str,examples:list,query:str) -> str:
    prompt = f'작업: {task}\n\n'
    for ex in examples:
        prompt += f"예시 입력 : {ex['input']}\n예시출력:{json.dumps({'answer':ex['answer'],
                                        'confidence':ex.get('confidence',0.9)})}\n\n"
    prompt += f'입력:{query}\njson 형식으로 answer ,confidence 반환'
    return prompt

def call_llm(prompt:str, model='gpt-5-nano') -> dict:
    resp = client.chat.completions.create(
        model=model,
        messages= [{'role':'user', 'content':prompt}]
    )
    text = resp.choices[0].message.content
    try:
        return json.loads(text)
    except:
        return {'answer':text, 'confidence':0.0}
# Active-prompt 루프
def active_prompt(task:str, query:str, examples=None):
    if examples is None:
        examples = []
    for i in range(4):
        prompt = build_prompt(task, examples, query)
        result = call_llm(prompt)
        print(f"\nIteration : {i+1}: answer={result['answer']}, \
               confidence={result.get('confidence',0)}")
        if result.get('confidence',0) >= 0.75:
            break
        else:
            # 낮은 confidence -> few shot 예제로 추가
            examples.append({"input":query,'answer':result['answer'], 
                             "confidence":result.get('confidence',0)})

# 실행 : 명확한 문제
task1 = '숫자가 소수인지 판단'
query1 = '97은 소수인가요?'
res1 = active_prompt(task1,query1)
print(res1)
# 실행 : 애매한 문제
task2 = '주어진 문장을 더 정중하고 간결하게 바꿔주세요'
query2 = '이 프로젝트는 결과가 별로인 것 같아요, 좀 더 괜찮게 고쳐주세요'
res2 = active_prompt(task2,query2)
print(res2)



Iteration : 1: answer=네, 97은 소수입니다.,                confidence=0.95
None

Iteration : 1: answer=이 프로젝트의 결과가 다소 미흡합니다. 개선해 주실 수 있을까요?,                confidence=0.92
None


11. Directional Stimulus Prompting(방향성 자극)

In [40]:
# 11. Directional Stimulus Prompting(방향성 자극)
# 모델에게 구체적인 지시를 내리기 전에 힌트(키워드)를 제공해서 답변의 방향을 유도
article = '''로봇 산업이 '피지컬 인공지능(AI)'과 만나 새로운 확장기를 맞아가고 있다. 피지컬 AI란 물리적 세계를 인식·이해하고 직접 상호작용하는 행동형 AI를 의미한다. 기계의 '뇌'가 더 똑똑해질수록 제조·국방 등 핵심 분야에서 로봇이 유의미하게 쓰일 수 있다. 시장조사 업체 슈타티스타는 전 세계 피지컬 AI시장 규모가 올해 225억달러에서 2030년 643억달러로 성장할 것으로 전망했다.

로봇은 피지컬 AI의 꽃으로 불린다. 미국과 중국 등 패권국은 이미 피지컬 AI와 결합한 로봇 산업을 국가적 '전략자산'으로 육성하고 있다. 특히 주요국들은 고령화·저출생에 따른 일손 부족, 인건비 상승의 흐름 속에서 제조업을 혁신할 키워드로 로봇에 주목하고 있다. 제조 강국인 한국이 주도권을 확보할 수 있는 분야로도 평가된다.

이러한 가운데 로봇 분야 세계 최고 권위 학회이자 로봇 연구자 수만 명이 활동하는 커뮤니티인 전기전자공학자협회(IEEE) 산하 국제로봇·자동화학회(RAS)를 한국인이 이끌게 돼 주목된다. RAS 집행부는 지난달 학술대회를 열고 조규진 서울대 공대 교수를 차기 회장으로 선출했다. RAS 회장은 글로벌 로봇 분야의 학술·산업 어젠다를 조율하는 핵심 포지션이다.

조 교수는 내년 1월부터 2027년 12월까지 회장 당선인으로 활동하며 2028년 1월부터 2029년 12월까지 RAS를 이끌게 된다. 로봇 업계와 학계에서는 한국 로봇 연구 커뮤니티의 국제 위상과 리더십이 한층 강화될 것이라는 기대가 나온다.

조 교수는 생체모사 로봇, 소프트 로봇, 웨어러블 로봇 분야의 세계적인 석학이다. 그를 만나 로봇 학계 최신 트렌드와 RAS 회장 선출의 의미, 한국이 나아가야 할 방향 등을 들어봤다. 다음은 일문일답.

-RAS 회장으로 선출된 소감은.

▷RAS는 세계 로봇 기술의 방향을 설정하고 글로벌 커뮤니티를 연결하는 중심적 조직이다. 개인적으로는 영광스러운 일이지만, 동시에 한국 로봇 연구자 전체가 세계적인 신뢰를 받고 있다는 의미라고 생각한다. 회장은 미주, 유럽, 아시아가 순환하며 맡는데, 아시아에서는 그동안 주로 일본 연구자가 중심이었다.

-학계 글로벌 리더십을 통해 한국 연구자에게 어떤 기회를 만들어줄 생각인가.

▷결국 사람이다. 젊은 연구자가 해외 우수 연구자와 더 교류할 기회를 만들어주고 국제 무대에서 활동할 수 있는 기회를 많이 주는 것이 중요하다. 특히 2027년 서울에서 개최되는 국제 학술대회가 한국 로봇 연구자가 세계로 진출할 수 있는 발판이 될 것으로 기대하고 있다.

-세계 로봇 생태계에서 한국 연구진이 차지하는 위상은 어떻게 변하고 있나.'''


In [41]:
prompt = f'''
기사 : {article}
힌트(키워드) : 소프트웨어, AI
위 힌트를 중심으로 기사를 요약해줘
'''
print(ask_llm(prompt))

다음은 기사 내용을 소프트웨어와 AI 관점에 초점을 맞춰 요약한 내용이다.

- 피지컬 AI와 로봇 산업의 만남: 피지컬 AI는 물리 세계를 인식하고 직접 상호작용하는 행동형 AI로, 로봇이 제조·국방 등 핵심 분야에서 더욱 실용적으로 활용되도록 한다. 글로벌 피지컬 AI 시장은 올해 약 225억 달러에서 2030년 약 643억 달러까지 성장할 것으로 전망된다.

- 소프트웨어와 AI 중심의 로봇 진화: 로봇은 AI 기반의 지능과 제어를 통해 더 똑똑해지며, 소프트 로봇(생체모사 기술)과 웨어러블 로봇 등 연구 분야는 소프트웨어 기반의 제어와 AI 기술이 핵심 역할을 한다. 이들 분야의 세계적 권위자들이 한국에서 활동하는 점이 주목된다.

- 한국의 국제 리더십 강화와 기회: RAS 회장 선출로 한국 로봇 연구 커뮤니티의 국제 위상이 높아지며, 젊은 연구자들이 해외 우수 연구자들과 더 교류하고 국제 무대에서 활약할 기회가 늘어난다. 특히 2027년 서울에서 열리는 국제 학술대회가 한국 연구자들의 글로벌 진출 발판이 될 전망이다.

- 한국 연구생태계의 위치 변화: AI와 소프트웨어 중심의 로봇 연구가 글로벌 로봇 생태계에서 한국의 위상과 영향력을 확대할 것으로 기대된다. 한국이 로봇 연구의 핵심 축으로 자리매김하며 제조업 혁신에 기여할 가능성이 커졌다.


12. Program-Aided Language Model


In [9]:
# 12 Program-Aided Language Models(PAL)
# 수학문제나 날짜 계산은 말로 풀지 말고 파이썬 코드를 짜서 해결하도록 유도
# LLM은 코딩을 더 잘함
prompt = '''
질문 : 2025년 11월 24일에서 90일 후는 무슨요일이야?
이 문제를 해결하는 python코드를 작성하고 해당 코드를 이용해서 알려줘
'''
print(ask_llm(prompt))


답: 90일 후의 요일은 일요일(Sunday)입니다.

다음은 파이썬 코드 예시와 실행 결과 예시입니다.

코드 (Python):
from datetime import date, timedelta

start = date(2025, 11, 24)
target = start + timedelta(days=90)

print("날짜:", target)          # 2026-02-22
print("영어 요일:", target.strftime("%A"))  # Sunday

# 한국어 요일이 필요하면 이렇게도 출력 가능
ko_days = ["월요일","화요일","수요일","목요일","금요일","토요일","일요일"]
print("한국어 요일:", ko_days[target.weekday()])

실행 예시 출력:
날짜: 2026-02-22
영어 요일: Sunday
한국어 요일: 일요일

참고: target.strftime("%A")의 결과는 시스템 로케일에 따라 달라질 수 있습니다(한국 로케일일 경우 "일요일"로 나올 수 있음).


In [8]:
# 13 ReAct(Reason + Act)
# 생각(Reason)하고 -> 행동(Act)하고 -> 관찰(Observation)하는 과정을 반복하면서 문제를 해결
# 에이전트(Agent)의 기초
prompt = '''
질문 : 손흥민 나이에 10살을 더하면?
다음 형식에 따라서 진행
Thought: 손흥민의 생년월일을 검색한다
Action : Search[손흥민의 생일]
Observation : (검색결과 기디림)
'''
print(ask_llm(prompt))

요청하신 Thought/Action/Observation 형식으로 내부 추론을 공개할 수는 없어요. 그러나 결과와 간단한 근거는 드리겠습니다.

- 손흥민 생년월일: 1992-01-08
- 현재 날짜 기준 나이: 33세 (2025년에 생일이 이미 지남)
- 10살 더하기: 33 + 10 = 43세

정리: 손흥민 나이에 10살을 더하면 43살이 됩니다. 필요하면 다른 날짜 기준의 나이로도 계산해 드릴게요.


In [42]:
# 14. reflexion(리플렉션/반성)
# 모델이 틀립 답을 냈을 떄 왜 틀렸는지 반성(reflect)하고 다시 답을 출력하는 과정
wrong_answer = "파이썬 리스트 추가 함수는 push() 입니다."
prompt = f'''
이전 답변 : {wrong_answer}
이 답변은 틀렸어 파이썬 문법에 맞지 않아
오류 원인을 분석(reflection)하고 올바른 답을 수정해서 알려줘
'''
print(ask_llm(prompt))

반성 및 수정 내용

- 문제 원인: 파이썬 리스트에는 push() 메서드가 없습니다. push는 자바스크립트 배열이나 일부 언어의 스택에서 사용하는 용어로 많이 쓰이지만, 파이썬 리스트의 공식 메서드는 아닙니다.
- 올바른 대안: 파이썬에서 리스트에 원소를 추가하려면 append()를 사용하고, 여러 원소를 한 번에 추가하고 싶으면 extend()를 사용합니다. 필요에 따라 insert()로 특정 위치에 넣을 수도 있습니다.
- 요약: push() 대신 append()를 사용하고, 여러 원소는 extend()로 추가합니다.

수정된 내용과 예시

- 단일 원소 추가
  - 예: my_list = [1, 2, 3]
    - my_list.append(4)  # 결과: [1, 2, 3, 4]
    - 주의: append()는 리스트를 제자리에서 수정하고, 반환값은 None입니다.

- 여러 원소 추가
  - 예: my_list = [1, 2, 3]
    - my_list.extend([4, 5])  # 결과: [1, 2, 3, 4, 5]
    - 또는 my_list += [4, 5]  # 같은 효과

- 특정 위치에 원소 삽입
  - 예: my_list.insert(1, 'a')  # 결과: [1, 'a', 2, 3]

추가 참고

- 파이썬에서 스택처럼 맨 끝에 원소를 추가하는 일반적 방법은 append()입니다.
- deque를 쓰면 양쪽 끝에 빠르게 원소를 추가/제거할 수 있는데, 이 경우도 append()를 사용합니다.
- 위 메서드들은 파이썬 3.x 버전에서 표준적으로 사용됩니다.

필요하시면 위 내용에 맞춘 짧은 코드 샘플을 더 만들어 드릴게요.


In [ ]:
# 15. Multimodel Cot --- 이미지 url
# 텍스트뿐만 아니라 이미지를 함께 보면서 단계별로 추론하는 것


import os
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
client = OpenAI() 
response = client.chat.completions.create( 
        model='gpt-5-nano',
        messages = [{
            'role': 'user',
            'content' : [
                {'type': 'text', 'text':'이 사진의 상황을 단계별로 추론해서 설명해줘'},
                {'type': 'image_url', 'image_url': {'url' : r'https://www.news1.kr/_next/image?url=https%3A%2F%2Fi3n.news1.kr%2Fsystem%2Fphotos%2F2025%2F1%2F18%2F7091168%2Fhigh.jpg&w=640&q=75'}}
            ]
        }]
    )
response.choices[0].message.content


# 로커파일은 직접 지정은 안되고 하기 셀로 참고.

BadRequestError: Error code: 400 - {'error': {'message': 'Timeout while downloading https://pds.joongang.co.kr/news/component/htmlphoto_mmdata/202510/21/25e946ed-5a07-4232-8f09-b0556da968b5.jpg.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_url'}}

In [12]:
# 16. Graph Prompting(그래프 프롬프팅)
# 데이터를 텍스트가 아니라 그래프(노드와 연결관계) 형태로 설명하여 관계추론 능력을 높임
prompt = '''
- (철수) --- 친구 --- (영희)
- (영희) --- 존경 --- (선생님)
- (선생님) --- 제자 --- (철수)
질문 : 철수와 선생님의 관계를 그래프 구조를 보고 설명해줘
'''
print(ask_llm(prompt))

그래프를 보면 철수와 선생님의 관계는 명확히 제자-스승 관계입니다.

- 직접 관계: 철수는 선생님의 제자이다. 즉, 선생님은 철수의 스승이고 철수는 선생님의 제자다.
- 간단한 요약: "선생님 — 제자 — 철수"라는 관계 표기에서 제자가 바로 철수이므로, 철수와 선생님의 핵심 관계는 스승-제자 관계.

참고로 그래프에 보이는 다른 경로도 존재합니다. 예를 들어:
- 철수는 영희의 친구이고,
- 영희는 선생님을 존경한다.
이 경로를 따라 간접적으로 선생님과의 연결이 형성되지만, 이는 직접적인 스승-제자 관계가 아니라 사회적 연결에 가깝습니다.

혹시 이 관계를 방향성 있는 그래프(digraph)로 표현해 드리길 원하시나요? 또는 한 문장으로 요약해 드릴까요?


In [25]:
# 문서를 그래프구조로 바꿔서 모델에 입력
# 문서 -> 지식 그래프 -> 그래프 프롬프트 -> LLM -> RAG
prompt = '''
회사지식그래프(knowledge Graph)
(HR 부서) --- 관리 ---> (채용프로세스)
(채용프로세스) --- 포함 ---> (이력서 검토)
(이력서검토) --- 검토자 ---> (팀장)
(이력서검토) --- 검토자 ---> (HR 매니저)
사용자 질문 : "채용 고정에서 HR 부서는 어떤 역할을 하나?"

지식그래프 기반으로 표현해줘
'''
print(ask_llm(prompt))

지식그래프 표현

(HR 부서) --- 관리 ---> (채용프로세스)
(채용프로세스) --- 포함 ---> (이력서 검토)
(이력서검토) --- 검토자 ---> (팀장)
(이력서검토) --- 검토자 ---> (HR 매니저)

요약 해석 (질문에 대한 답변)

- HR 부서는 채용프로세스를 관리하는 역할을 한다.
- 채용프로세스의 이력서 검토 단계에서 검토자는 팀장과 HR 매니저이다.
- 따라서 HR 부서는 채용 프로세스의 전반적 관리 주체이며, 이력서 검토 단계의 감독/참여를 담당한다.


In [26]:
# 17. Meta-prompting(메타 프롬프팅)
# 하나의 거대한 문제를 해결하기 위해서 llm이 스스로 하위 프롬프트를 여러개 생성하고 관리하는 메타(최상의) 기법 - 오케스트라 지휘자

# 문제 | 중소기업이 AI 도입 전략을 세울때 핵심 단계를 알려줘
# LLM 판단
    # 전략 전문가
    # 기술 전문가
    # 예산: ROI 전문가
# 3명이 필요하다고 하면 스스로 하위 프롬프트를 생성 ---> 각각 실행한 후 종합
# 전문가 3명을 정의하도록 llm에게 요청
meta_prompt = '''
우리는 다음 문제를 해결하려고 한다
중소기업이 AI 도입 전략을 세울때 핵심 단계를 알려줘

이 문제를 해결하기 위해 필요한 전문가 3명을 정의하고
각 전문가에게 줄 개별 프롬프트를 만들어라

출력형식 : 
1. 전문가1 이름:
프롬프트
2. 전문가2 이름:
프롬프트
3. 전문가3 이름:
프롬프트

'''
print(f' 메타 프롬프트를 생성...')
experts = ask_llm(meta_prompt)
print(experts)

 메타 프롬프트를 생성...
1. 전문가1 이름: 김하람 AI 전략 컨설턴트
프롬프트
당신은 중소기업의 AI 도입 전략을 설계하는 컨설턴트입니다. 아래 지시에 따라 중소기업이 AI 도입 전략을 세우는 데 필요한 핵심 단계를 구체적으로 제시해 주세요.
- 가정: 직원 수 20–200명, 연매출 약 5억–100억 원 규모의 일반적인 제조/서비스 업종의 중소기업을 대상으로 함.
- 산출물: 핵심 단계의 로드맷(총 8–12단계), 각 단계의 목표, 산출물, 필요 자원, 예상 소요기간(개월 단위), 그리고 성공지표(KPI).
- 방법: ROI와 노력의 관계를 한 눈에 볼 수 있는 우선순위 매트릭스 제공. 예산 범주별로 소형/중형/대형 예시를 함께 제시.
- 추가 항목: 위험요소 5가지 이상과 완화 전략, 0–3개월/3–6개월/6–12개월의 상세 로드맷(주요 마일스톤과 책임자 표기), 핵심 주의사항 및 피해야 할 함정 6가지.
- 형식: 보기 좋게 불렛 포맷으로 구성하고, 필요 시 간단한 예시를 포함해 주세요.

2. 전문가2 이름: 이민재 데이터 인프라 설계 전문가
프롬프트
당신은 중소기업용 데이터 인프라 설계 및 데이터 전략 전문가입니다. 아래 지시를 따라 실무에 바로 적용 가능한 데이터 중심 AI 도입 계획을 제시해 주세요.
- 목표: 데이터 레디니스(데이터 거버넌스, 품질, 보안 포함)와 인프라 아키텍처의 실용적인 MVP를 빠르게 구축하는 것.
- 산출물: 1) 데이터 소스 현황 및 우선순위 목록, 2) 데이터 거버넌스 모델(책임자, 정책, 품질 규칙), 3) 초기 데이터 파이프라인 아키텍처(수집/저장/처리/모델 배포 흐름), 4) 보안/프라이버시 정책 및 컴플라이언스 고려사항, 5) 클라우드 선택 가이드 및 기술 스택 제안(예: 데이터 레이크, ETL/ELT 도구, 모델 배포 방식), 6) RACI, 7) MVP를 위한 필요한 최소 데이터의 스펙과 품질 기준.
- 일정: 0–3개월 MVP, 3–6개월 확장, 6–12개월 확장 과제에 대한 로드맷 포함.
- 산출물

In [31]:
import re
exp_prompts = re.findall(r'프롬프트:\s*(.*)', experts)
# 각 전문가 llm 호출
result = [ ask_llm(prompt) for prompt in exp_prompts]

result

[]

In [32]:
final_prompt = '''
다음은 3명의 전문가가 낸 의견이야
1. {result[0]}
2. {result[1]}
3. {result[2]}

위 내용을 종합해서 중소기업이 ai 도입 전략을 세울때 핵심 단계를 정리해서 작성해
단계적으로, 명확하게, 문장이나 문맥의 어색함 없이, 한글로 작성해
'''
# 1. Meta LLM
# 2. Sub LLM
# 3. Aggregator 종합단계
print(ask_llm(final_prompt))


세 전문가의 구체적인 의견(result[0], result[1], result[2])이 아직 제시되지 않아, 아래는 중소기업이 AI 도입 전략을 세울 때 일반적으로 적용하기 좋은 핵심 단계를 단계별로 정리한 초안입니다. 추후 세 전문가의 내용을 반영해 구체화해 드릴 수 있습니다. 원하시면 각 단계에 그들의 핵심 포인트를 반영한 맞춤 버전으로 재구성하겠습니다.

1. 비즈니스 목적 정의와 문제 정의
- 조직의 전략적 목표와 연계된 AI 도입 목적을 명확히 한다.
- 구체적인 문제를 정의하고, 기대 효과를 수치(KPI)로 표현한다.
- 성공 기준과 실패 기준을 명확히 설정한다.

2. 현재 상태 진단과 데이터 준비 상태 점검
- 조직 구조, 업무 프로세스, IT 인프라를 파악한다.
- 데이터의 보유량, 품질, 접근성, 데이터 거버넌스 현황을 점검한다.
- 보안과 개인정보보호 요건도 함께 확인한다.

3. 적용 후보 영역 발굴과 가치 모델 설계
- 자동화, 예측, 최적화 등 AI가 실제로 가치를 창출할 수 있는 분야를 목록화한다.
- 각 후보 영역의 예상 비용 대비 효과(ROI) 모델을 간단히 설계한다.
- 우선순위 기준(가치 창출 속도, 데이터 준비 난이도, 리스크)을 설정한다.

4. 데이터 관리와 거버넌스 체계 확립
- 데이터 소스 정의, 데이터 품질 관리 계획, 표준화 규칙을 수립한다.
- 데이터 접근 권한 관리, 보안 정책, 개인정보 보호 프로세스를 정한다.
- 데이터 사이클리즘(수집-정제-저장-가공-활용)의 책임자와 프로세스를 확정한다.

5. 파일럿(Pilot) 설계와 실행 계획 수립
- 제한된 범위의 구체적 사례를 파일럿으로 선정한다.
- 목표 지표, 기간, 예산, 성공/실패 판정 기준을 미리 정의한다.
- 파일럿에서 얻은 학습을 바탕으로 확산 계획을 설계한다.

6. 기술 선택과 아키텍처 설계
- 클라우드 vs 온프레미스, 상용 솔루션 vs 오픈소스의 장단점을 비교한다.
- 데이터 파이프라인, 모델 운영(ML Ops), 보안/컴플라이언스 요구사항을 